# Portfolio Optimization 

## 1.- Efficient Frontier

In [ ]:
# source: https://towardsdatascience.com/efficient-frontier-in-python-detailed-tutorial-84a304f03e79

In [2]:
# Data
import pandas as pd
from pandas_datareader import data as pdr #Economic and financial dataframes
import numpy as np 
import yfinance as yf #Yahoo finance
yf.pdr_override()

from wasabi import msg #Command lines
from sklearn.linear_model import ElasticNet
from sklearn.metrics import auc, roc_curve, confusion_matrix
from tqdm import tqdm  #Progress bar
import datetime 
import shap #Models explanation

## Data

In [96]:
tickers = ['AAPL', 'GOOGL', 'GOOG', 'MSFT', 'AMZN', 'FB',
           'BRK-A', 'BRK-B', 'TSLA', 'JPM', 'JNJ', 'NVDA',
           'BAC', 'WMT', 'V', 'PG', 'MA', 'HD', 'UNH', 'DIS',
           'CMCSA', 'XOM', 'KO', 'VZ', 'PFE', 'MRK', 'BMY', 'NKE',
           'CVX', 'PEP', 'INTC', 'CSCO', 'NFLX', 'ADBE', 'ORCL', 'CRM',
           'T', 'TMUS', 'C', 'ABT', 'TMO', 'ABBV', 'HON', 'MCD', 'AXP', 'PM',
           'BA', 'NEE', 'MDT', 'PYPL', 'GE', 'COST', 'TXN', 'DHR', 'WFC', 'F',
           'BAC-WT', 'UNP', 'ACN', 'LMT', 'CME', 'UPS', 'LOW', 'CVS', 'ADP', 'FIS',
           'AMGN', 'SBUX', 'RTX', 'USB', 'MMM', 'BMY-RT', 'BLK', 'LLY', 'SPGI', 'DE',
           'PLD', 'KO-PC', 'LIN', 'SLB', 'UPS-PA', 'IBM', 'SPG', 'EQIX', 'CMI', 'FDX',
           'CCI', 'ZTS', 'BDX', 'TJX', 'ISRG', 'D', 'AON', 'MMC', 'JCI', 'PNC', 'SCHW',
           'KMI', 'APD', 'ANTM']

In [101]:
tickers = tickers
data = pdr.get_data_yahoo(tickers, start = '2018-03-01', end = '2023-03-01').dropna(axis=1)
data.tail()

[*********************100%***********************]  100 of 100 completed

6 Failed downloads:
- ANTM: No timezone found, symbol may be delisted
- KO-PC: No timezone found, symbol may be delisted
- BAC-WT: No timezone found, symbol may be delisted
- FB: No timezone found, symbol may be delisted
- BMY-RT: No timezone found, symbol may be delisted
- UPS-PA: No timezone found, symbol may be delisted


Adj Close                                      \
                           AAPL        ABBV         ABT         ACN   
Date                                                                  
2023-02-22 00:00:00  148.910004  150.669998  102.690002  269.630005   
2023-02-23 00:00:00  149.399994  152.070007  102.550003  271.230011   
2023-02-24 00:00:00  146.710007  152.710007  100.459999  266.529999   
2023-02-27 00:00:00  147.919998  154.279999   99.769997  266.250000   
2023-02-28 00:00:00  147.410004  153.899994  101.720001  265.549988   

                                                                    \
                           ADBE         ADP        AMGN       AMZN   
Date                                                                 
2023-02-22 00:00:00  348.720001  221.674591  236.160004  95.790001   
2023-02-23 00:00:00  347.019989  221.525436  237.619995  95.820000   
2023-02-24 00:00:00  320.540009  220.332199  233.660004  93.500000   
2023-02-27 00:00:00  322.320007  220.352097  234.449997  93.760002   
2023-02-28 00:00:00  323.950012  218.582123  231.660004  94.230003   

                                             ...   Volume                     \
                            AON         APD  ...      UNH       UNP      UPS   
Date                                         ...                               
2023-02-22 00:00:00  304.869995  278.290009  ...  2982800   5816000  2376900   
2023-02-23 00:00:00  304.480011  279.720001  ...  2511800   3125400  2341200   
2023-02-24 00:00:00  302.970001  280.980011  ...  2940400   3371400  2674400   
2023-02-27 00:00:00  302.589996  285.420013  ...  3006200  12140400  2925400   
2023-02-28 00:00:00  304.049988  285.980011  ...  3902100   6091200  3703000   

                                                                              \
                         USB        V        VZ       WFC      WMT       XOM   
Date                                                                           
2023-02-22 00:00:00  4583300  4621800  15086800  13402900  9948000  13973800   
2023-02-23 00:00:00  4319300  5533800  16738200  14783600  7340400  11289400   
2023-02-24 00:00:00  4141500  4414400  16431500  16040300  6874500  14351000   
2023-02-27 00:00:00  4260200  4255300  14210900  15468200  5549100  11710500   
2023-02-28 00:00:00  6209100  5385400  16685300  18627200  6018000  18107000   

                              
                         ZTS  
Date                          
2023-02-22 00:00:00  2103800  
2023-02-23 00:00:00  1769000  
2023-02-24 00:00:00  1843200  
2023-02-27 00:00:00  1183700  
2023-02-28 00:00:00  2935200  

[5 rows x 564 columns]

## Prices

In [102]:
prices = data['Adj Close']
prices.tail()

,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMGN,AMZN,AON,APD,...,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-22 00:00:00,148.910004,150.669998,102.690002,269.630005,348.720001,221.674591,236.160004,95.790001,304.869995,278.290009,...,487.154907,190.097687,176.610001,47.540001,220.020004,39.279999,46.009998,143.651230,109.730003,166.429993
2023-02-23 00:00:00,149.399994,152.070007,102.550003,271.230011,347.019989,221.525436,237.619995,95.820000,304.480011,279.720001,...,489.944946,192.024704,179.940002,47.720001,221.130005,38.529999,46.250000,141.509995,110.739998,169.500000
2023-02-24 00:00:00,146.710007,152.710007,100.459999,266.529999,320.540009,220.332199,233.660004,93.500000,302.970001,280.980011,...,482.611053,192.720001,180.149994,47.639999,219.550003,38.740002,46.619999,141.888458,110.750000,166.330002
2023-02-27 00:00:00,147.919998,154.279999,99.769997,266.250000,322.320007,220.352097,234.449997,93.760002,302.589996,285.420013,...,481.604645,212.169998,182.229996,47.580002,220.350006,38.880001,46.779999,140.862656,110.550003,165.470001
2023-02-28 00:00:00,147.410004,153.899994,101.720001,265.549988,323.950012,218.582123,231.660004,94.230003,304.049988,285.980011,...,474.250854,207.279999,182.490005,47.730000,219.940002,38.810001,46.770000,141.549850,109.910004,167.000000


## Returns

In [103]:
# Daily, Monthly and yearly returns
''' daily_return = prices.pct_change(1) # 1 for ONE DAY lookback
    monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
    annual_return = prices.pct_change(252) # 252 for ONE YEAR lookback
'''
returns = prices.pct_change(1).dropna()
returns.tail()

,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMGN,AMZN,AON,APD,...,UNH,UNP,UPS,USB,V,VZ,WFC,WMT,XOM,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-22 00:00:00,0.002896,-0.008750,-0.009262,0.001783,0.005826,-0.007789,-0.008731,0.012793,0.000624,0.002811,...,-0.004926,-0.012232,-0.010367,-0.003145,-0.002720,0.002041,-0.004974,-0.020973,-0.012953,-0.020078
2023-02-23 00:00:00,0.003291,0.009292,-0.001363,0.005934,-0.004875,-0.000673,0.006182,0.000313,-0.001279,0.005138,...,0.005727,0.010137,0.018855,0.003786,0.005045,-0.019094,0.005216,-0.014906,0.009204,0.018446
2023-02-24 00:00:00,-0.018005,0.004209,-0.020380,-0.017329,-0.076307,-0.005386,-0.016665,-0.024212,-0.004959,0.004505,...,-0.014969,0.003621,0.001167,-0.001676,-0.007145,0.005450,0.008000,0.002674,0.000090,-0.018702
2023-02-27 00:00:00,0.008248,0.010281,-0.006868,-0.001051,0.005553,0.000090,0.003381,0.002781,-0.001254,0.015802,...,-0.002085,0.100924,0.011546,-0.001259,0.003644,0.003614,0.003432,-0.007230,-0.001806,-0.005170
2023-02-28 00:00:00,-0.003448,-0.002463,0.019545,-0.002629,0.005057,-0.008032,-0.011900,0.005013,0.004825,0.001962,...,-0.015269,-0.023048,0.001427,0.003153,-0.001861,-0.001800,-0.000214,0.004878,-0.005789,0.009246


## Annualized mean return and covariances

In [104]:
annual_returns = ((1+returns.mean())**(252))-1
cov = returns.cov()*252

print("The annualized mean returns for each asset are: ")
print(annual_returns)

The annualized mean returns for each asset are: 
AAPL    0.362086
ABBV    0.157086
ABT     0.175741
ACN     0.176549
ADBE    0.176068
          ...   
VZ      0.024008
WFC     0.059169
WMT     0.148242
XOM     0.204017
ZTS     0.212427
Length: 94, dtype: float64


In [105]:
print("The covariance for each asset are: ")
print(cov)

The covariance for each asset are: 
          AAPL      ABBV       ABT       ACN      ADBE       ADP      AMGN  \
AAPL  0.111937  0.032893  0.049253  0.062005  0.085984  0.056338  0.038330   
ABBV  0.032893  0.075755  0.034665  0.034340  0.035324  0.033254  0.034718   
ABT   0.049253  0.034665  0.071161  0.047468  0.055368  0.044835  0.037329   
ACN   0.062005  0.034340  0.047468  0.082005  0.070516  0.058869  0.033886   
ADBE  0.085984  0.035324  0.055368  0.070516  0.142147  0.061729  0.040216   
...        ...       ...       ...       ...       ...       ...       ...   
VZ    0.019201  0.019110  0.021542  0.023174  0.015818  0.024962  0.020489   
WFC   0.050786  0.035858  0.037781  0.057608  0.045351  0.057733  0.034132   
WMT   0.029580  0.017317  0.023304  0.024733  0.030389  0.024591  0.022729   
XOM   0.038684  0.030820  0.024746  0.041433  0.030618  0.044809  0.025136   
ZTS   0.055418  0.033567  0.048424  0.053648  0.062785  0.049064  0.032365   

          AMZN       AON   

## Sample for efficient frontier

In [106]:
# https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation

In [115]:
# Assets per portfolio
n_assets = 5

# Generated portfolios
n_portf = 1000

# List of mean and variance, weights and tickers
mean_var = []
weights_list = []
tickers_list = []

for i in tqdm(range(n_portf)):
    next_i = False
    while True:
        # Choose assets randomly without replacement
        assets = np.random.choice(list(returns.columns), n_assets, replace=False)
        
        # Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights / sum(weights)
        
        # Compute portfolio returns and variance
        portf_return = 0
        portf_var = 0
        for i in range(len(assets)):
            portf_return += weights[i] * annual_returns.loc[assets[i]]
            for j in range(len(assets)):
                portf_var += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
        
        # Skip dominated portfolios
        for r, v in mean_var:
            if r > portf_return and v < portf_var:
                next_i = True
                break
        if next_i:
            break
        
        # Add the mean/variance pairs, weights and tickers to lists
        mean_var.append([portf_return, portf_var])
        weights_list.append(weights)
        tickers_list.append(assets)
        break


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2680.16it/s]


In [116]:
len(mean_var)

48

In [117]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

# Plot risk vs return
mean_var = np.array(mean_var)

# Risk free rate
Rf = 0.0351 # Ten year treassury bond rate

fig = go.Figure()
fig.add_trace(go.Scatter(x = mean_var[:,1]**0.5, y = mean_var[:,0],
                          marker =dict(color=(mean_var[:,0]-Rf)/(mean_var[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ),
              mode = 'markers',
              text = [str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]
              )
)
fig.update_layout(template='plotly_white',xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
fig.update_xaxes(range=[0.18, 0.35])
fig.update_yaxes(range=[0.05,0.29])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))
              



In [118]:
tickers_list

[array(['ZTS', 'DIS', 'KMI', 'NEE', 'T'], dtype='<U5'),
 array(['TMO', 'LOW', 'WMT', 'MSFT', 'BDX'], dtype='<U5'),
 array(['BMY', 'TSLA', 'PYPL', 'UPS', 'F'], dtype='<U5'),
 array(['BMY', 'PM', 'LLY', 'BRK-B', 'TSLA'], dtype='<U5'),
 array(['D', 'LLY', 'HON', 'CMI', 'ADBE'], dtype='<U5'),
 array(['ZTS', 'MA', 'MSFT', 'UNP', 'NEE'], dtype='<U5'),
 array(['UNP', 'AAPL', 'PFE', 'ABBV', 'PEP'], dtype='<U5'),
 array(['VZ', 'COST', 'SLB', 'DIS', 'PLD'], dtype='<U5'),
 array(['TSLA', 'CMI', 'DE', 'PLD', 'SPGI'], dtype='<U5'),
 array(['IBM', 'TSLA', 'V', 'ABBV', 'AON'], dtype='<U5'),
 array(['CMCSA', 'TSLA', 'MDT', 'BRK-A', 'BRK-B'], dtype='<U5'),
 array(['AON', 'UNH', 'MRK', 'BRK-A', 'HD'], dtype='<U5'),
 array(['INTC', 'CMI', 'PFE', 'LLY', 'TSLA'], dtype='<U5'),
 array(['BDX', 'HON', 'GOOG', 'BMY', 'PM'], dtype='<U5'),
 array(['BDX', 'COST', 'SPGI', 'TSLA', 'ADBE'], dtype='<U5'),
 array(['SPGI', 'PFE', 'VZ', 'JNJ', 'CME'], dtype='<U5'),
 array(['PM', 'AMZN', 'VZ', 'COST', 'LMT'], dtype='<U5'